In [3]:
import ollama
import gradio as gr

model="llama2"

In [ ]:
# Chat function for Gradio using ollama
#Inputs that are a list

Hashtags= ["Include Hashtags", "Donot include hashtags"]
emojis = ["Include emojis", "Donot include emojis"]
def post_inputs(link, picture, Hashtags,emojis):
    user_prompt = f"Input any link that you like to include in the post: {link}\n"
    user_prompt += f"Input Picture that you like to include in the post: {picture}.\n"
    user_prompt += f"Do you want Hashtags in your post?: {Hashtags}.\n"
    user_prompt += f"Do you want to include Emojis in your post?: {emojis}."
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

def chat_llama2(user_prompt, history):
    system_message = "You are a professional LinkedIn post creator"


   


    messages = [
        {"role":"assistant","content":system_message},
        {"role":"user","content": user_prompt}
    ]

    result = ollama.chat(
        model="llama2",
        messages=messages,
        stream=True         # allows us to stream the resulting response instead of displaying the entire output at once
        )
    
    # streaming the response from the model
    # this prints out each output token form the model to have the resulting text displayed in real time
    response = ""
    for chunk in result:
        if 'message' in chunk and 'content' in chunk['message']:
            chunk_text = chunk['message']['content']
            response += chunk_text or ""
            yield response

In [23]:
import ollama
import gradio as gr
import os

# Chat function for generating LinkedIn posts
def chat_llama2(prompt, files):
    system_message = "You are a professional LinkedIn post creator. Create a professional LinkedIn post based on the user's inputs."

    # Prepare the input prompt, including file names if provided
    file_info = ""
    images = []  # Collect image file paths to display in the output
    if files:
        for file in files:
            file_info += f"File uploaded: {file.name}\n"
            # Check if the file is an image
            if file.name.lower().endswith((".png", ".jpg", ".jpeg", ".gif")):
                # Save the image to a temporary folder for display
                file_path = os.path.join("temp_images", file.name)
                os.makedirs("temp_images", exist_ok=True)  # Ensure the directory exists
                with open(file_path, "wb") as f:
                    f.write(file.encode("latin1"))  # Convert the string to bytes before writing
                images.append(file_path)

    user_prompt = f"{prompt}\n\n{file_info}"

    # Initialize the conversation history
    history = [
        {"role": "assistant", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    # Add uploaded images to the conversation history for display
    for image_path in images:
        history.append({"role": "user", "content": f'<img src="{image_path}" alt="Uploaded Image" style="max-width: 300px;">'})

    # Call the Ollama API
    result = ollama.chat(
        model="llama2",
        messages=history,
        stream=True  # Stream the response
    )

    # Stream the response and update the history
    for chunk in result:
        if "message" in chunk and "content" in chunk["message"]:
            chunk_text = chunk["message"]["content"]
            if history[-1]["role"] == "assistant":
                # Append to the last assistant message
                history[-1]["content"] += chunk_text
            else:
                # Create a new assistant message
                history.append({"role": "assistant", "content": chunk_text})
            yield history  # Yield the updated history

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# LinkedIn Post Generator")
    gr.Markdown("Enter the topic and optionally upload files to generate a professional LinkedIn post.")

    chatbot = gr.Chatbot(elem_id="chatbot", height=400, type="messages")

    with gr.Row():
        text_input = gr.Textbox(
            placeholder="Type the topic for your LinkedIn post here...",
            label="Post Topic",
            interactive=True
        )
        file_input = gr.File(
            file_types=["image", "application/pdf", "text"],
            label="Upload Files (Optional)",
            file_count="multiple"
        )

    submit_button = gr.Button("Generate Post")

    # Bind inputs and outputs
    submit_button.click(
        fn=chat_llama2,
        inputs=[text_input, file_input],
        outputs=chatbot
    )

# Launch the Gradio app
demo.launch()


* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.
